In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configuración del driver de Selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# Maximiza la ventana del navegador
driver.maximize_window()

# URL objetivo
url = "https://www.universidades.gob.es/listado-de-universidades/"

def scrape_universidades():
    # Abre la URL
    driver.get(url)
    wait = WebDriverWait(driver, 10)

    # Pulsar el botón de aceptar cookies
    try:
        cookies_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#moove_gdpr_cookie_info_bar > div > div > div.moove-gdpr-button-holder > button.mgbutton.moove-gdpr-infobar-allow-all.gdpr-fbo-0")))
        cookies_button.click()
    except Exception as e:
        print("No se pudo pulsar el botón de cookies: ", e)

    # Espera a que cargue la sección de comunidades autónomas
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "accordion")))

    # Encuentra los botones de las comunidades autónomas
    comunidades = driver.find_elements(By.CSS_SELECTOR, ".accordion .accordion-item .accordion-header button")

    data = []  # Lista para almacenar los datos

    for i, comunidad in enumerate(comunidades):
        try:
            # Hacer clic en cada comunidad autónoma
            ActionChains(driver).move_to_element(comunidad).click(comunidad).perform()

            # Esperar a que la sección se expanda
            time.sleep(1)  # Tiempo de espera para asegurar que cargue

            # Extraer el contenido HTML de la sección expandida
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            section = soup.select(f".accordion .accordion-item:nth-of-type({i + 1}) .accordion-body")

            if section:
                universidades = section[0].find_all("li")  # Encuentra las universidades dentro de la sección

                for universidad in universidades:
                    comunidad_nombre = comunidad.text.strip()
                    universidad_nombre = universidad.text.strip()
                    data.append({
                        "Comunidad Autónoma": comunidad_nombre,
                        "Universidad": universidad_nombre
                    })

        except Exception as e:
            print(f"Error procesando la comunidad {comunidad.text.strip()}: {e}")

    # Convertir los datos a un DataFrame
    df = pd.DataFrame(data)
    return df

# Ejecutar el scraping
df_universidades = scrape_universidades()

# Cerrar el navegador
driver.quit()

# Mostrar los resultados
df_universidades.to_csv("universidades_por_comunidad.csv", index=False)
print("Scraping completado. Datos guardados en 'universidades_por_comunidad.csv'.")
